In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime as dt

# Question 1: What is a Common Table Expression (CTE) in SQL?

- A Common Table Expression (CTE) is a named temporary result set in SQL that you can reference within a SELECT, INSERT, UPDATE, or DELETE statement. 
- CTEs are defined using the WITH clause and provide a way to create more readable and modular queries, especially for complex queries that involve multiple subqueries.

# Question 2: How do you define a CTE in SQL?

WITH cte_name (column1, column2, ...) AS (
    SELECT column1, column2, ...
    FROM your_table
    WHERE conditions
)

# Question 3: How is a CTE different from a temporary table in SQL?

- A CTE is a temporary result set that exists only for the duration of the query, while a temporary table is a physical table that is created and stored in the tempdb database. 
- CTEs are defined within the scope of a single query and are not stored persistently, whereas temporary tables persist until they are explicitly dropped or until the session ends.

# Question 4: What is the scope of a CTE in SQL?

- The scope of a CTE is limited to the query in which it is defined. 
- Once the query execution is complete, the CTE is discarded and not accessible anymore. 
- It cannot be referenced from outside the query in which it is defined.

# Question 5: How can you reference a CTE within a SQL query?

- You can reference a CTE within a SQL query by specifying its name after the WITH clause. 
- The CTE name acts as a derived table or a subquery that you can refer to in the main query.

**Example**

In [2]:
cnn = sqlite3.connect('database1.db')
cur = cnn.cursor()

In [3]:
cur.execute('''
CREATE TABLE Employee (
    EmployeeID INT PRIMARY KEY,
    Name VARCHAR(100),
    Job VARCHAR(50),
    Salary INT,
    DepartmentID INT,
    FOREIGN KEY (DepartmentID) REFERENCES Department(DepartmentID)
);
''')

In [4]:
cur.execute('''
INSERT INTO Employee (EmployeeID, Name, Job, Salary, DepartmentID) VALUES
(1, 'John Doe', 'Sales Representative', 50000, 1),
(2, 'Jane Smith', 'Marketing Specialist', 60000, 2),
(3, 'Mike Johnson', 'Financial Analyst', 70000, 3),
(4, 'Emily Davis', 'HR Manager', 55000, 4),
(5, 'Robert Brown', 'Sales Representative', 75000, 1),
(6, 'Alice White', 'Marketing Specialist', 80000, 2),
(7, 'Daniel Lee', 'Senior Financial Analyst', 90000, 3),
(8, 'Olivia Miller', 'HR Manager', 70000, 4),
(9, 'William Turner', 'Sales Representative', 55000, 1),
(10, 'Sophia Martin', 'Financial Analyst', 60000, 2);
''')

In [5]:
cur.execute('''
SELECT *
FROM Employee;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,EmployeeID,Name,Job,Salary,DepartmentID
0,1,John Doe,Sales Representative,50000,1
1,2,Jane Smith,Marketing Specialist,60000,2
2,3,Mike Johnson,Financial Analyst,70000,3
3,4,Emily Davis,HR Manager,55000,4
4,5,Robert Brown,Sales Representative,75000,1
5,6,Alice White,Marketing Specialist,80000,2
6,7,Daniel Lee,Senior Financial Analyst,90000,3
7,8,Olivia Miller,HR Manager,70000,4
8,9,William Turner,Sales Representative,55000,1
9,10,Sophia Martin,Financial Analyst,60000,2


In [6]:
cur.execute('''
-- Common Table Expression (CTE) definition
WITH DepartmentCTE AS (
    SELECT
        DepartmentID,
        AVG(Salary) AS AvgSalary
    FROM
        Employee
    GROUP BY
        DepartmentID
)

-- Main query using the CTE
SELECT
    e.EmployeeID,
    e.Name,
    e.Job,
    e.Salary,
    e.DepartmentID,
    d.AvgSalary
FROM
    Employee e
JOIN
    DepartmentCTE d ON e.DepartmentID = d.DepartmentID;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,EmployeeID,Name,Job,Salary,DepartmentID,AvgSalary
0,1,John Doe,Sales Representative,50000,1,60000.000000
1,2,Jane Smith,Marketing Specialist,60000,2,66666.666667
2,3,Mike Johnson,Financial Analyst,70000,3,80000.000000
3,4,Emily Davis,HR Manager,55000,4,62500.000000
4,5,Robert Brown,Sales Representative,75000,1,60000.000000
5,6,Alice White,Marketing Specialist,80000,2,66666.666667
6,7,Daniel Lee,Senior Financial Analyst,90000,3,80000.000000
7,8,Olivia Miller,HR Manager,70000,4,62500.000000
8,9,William Turner,Sales Representative,55000,1,60000.000000
9,10,Sophia Martin,Financial Analyst,60000,2,66666.666667


# Question 6: Retrieve a list of employees and their respective managers using a CTE.

In [7]:
cnn = sqlite3.connect('database2.db')
cur = cnn.cursor()

In [8]:
cur.execute('''
CREATE TABLE Employee (
    employee_ID INT PRIMARY KEY,
    employee_name VARCHAR(100),
    manager_id INT
);
''')

In [9]:
cur.execute('''
INSERT INTO Employee (employee_ID, employee_name, manager_id) VALUES
(1, 'Alice', 3),
(2, 'Bob', 3),
(3, 'Charlie', 4),
(4, 'David', NULL);
''')

In [10]:
cur.execute('''
SELECT *
FROM Employee;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,employee_ID,employee_name,manager_id
0,1,Alice,3.0
1,2,Bob,3.0
2,3,Charlie,4.0
3,4,David,NaN


In [11]:
cur.execute('''
WITH EmployeeWithManagerCTE AS(
    SELECT 
        e.employee_name, 
        m.employee_name AS manager_name 
    FROM 
        employee e
    LEFT JOIN 
        employee m ON e.manager_id = m.employee_id 
)
SELECT * 
FROM EmployeeWithManagerCTE;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,employee_name,manager_name
0,Alice,Charlie
1,Bob,Charlie
2,Charlie,David
3,David,None


# Question 7: Find the total sales for each product in the "Product" table. And use a CTE to achieve this.

In [12]:
cnn = sqlite3.connect('database3.db')
cur = cnn.cursor()

In [13]:
cur.execute('''
CREATE TABLE Product (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255)
);
''')

In [14]:
cur.execute('''
CREATE TABLE Sales (
    sale_id INT PRIMARY KEY,
    product_id INT,
    sale_date DATE,
    sale_amount INT,
    FOREIGN KEY (product_id) REFERENCES Product(product_id)
);
''')

In [15]:
cur.execute('''
INSERT INTO Product (product_id, product_name) VALUES
    (1, 'Product A'),
    (2, 'Product B'),
    (3, 'Product C');
''')

In [16]:
cur.execute('''
INSERT INTO Sales (sale_id, product_id, sale_date, sale_amount) VALUES
    (1, 1, '2023-01-05', 100),
    (2, 2, '2023-01-10', 150),
    (3, 1, '2023-01-15', 75),
    (4, 3, '2023-01-20', 200),
    (5, 2, '2023-01-25', 50);
''')

In [17]:
cur.execute('''
 SELECT *
 FROM Product;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

cur.execute('''
 SELECT *
 FROM Sales;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,product_id,product_name
0,1,Product A
1,2,Product B
2,3,Product C


,sale_id,product_id,sale_date,sale_amount
0,1,1,2023-01-05,100
1,2,2,2023-01-10,150
2,3,1,2023-01-15,75
3,4,3,2023-01-20,200
4,5,2,2023-01-25,50


In [18]:
cur.execute('''
WITH TotalSalesForEachProduct AS (
    SELECT 
        product_id, 
        SUM(sale_amount) AS TotalAmount
    FROM 
        Sales
    GROUP BY 
        product_id
)
SELECT 
    product_name, 
    TotalAmount
FROM 
    Product
JOIN 
    TotalSalesForEachProduct ON Product.product_id = TotalSalesForEachProduct.product_id
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,product_name,TotalAmount
0,Product A,175
1,Product B,200
2,Product C,200


# Question 8 :-
#### Use CTE
1. Display the names of all employees and their respective managers.
2. List all employees and their direct reports.
3. Calculate the number of levels in the employee hierarchy for each employee.

In [19]:
cnn = sqlite3.connect('database4.db')
cur = cnn.cursor()

In [20]:
cur.execute('''
CREATE TABLE Employee (
    employee_ID INT PRIMARY KEY,
    employee_name VARCHAR(100),
    manager_id INT
);
''')

In [21]:
cur.execute('''
INSERT INTO Employee (employee_ID, employee_name, manager_id) VALUES
(1, 'Alice', 3),
(2, 'Bob', 3),
(3, 'Charlie', 4),
(4, 'David', NULL);
''')

In [22]:
cur.execute('''
SELECT *
FROM Employee;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,employee_ID,employee_name,manager_id
0,1,Alice,3.0
1,2,Bob,3.0
2,3,Charlie,4.0
3,4,David,NaN


### Display the names of all employees and their respective managers.
# Or
### List all employees and their direct reports.

In [23]:
cur.execute('''
WITH EmployeeWithManagerCTE AS(
    SELECT 
        e.employee_name, 
        m.employee_name AS manager_name 
    FROM 
        employee e
    LEFT JOIN 
        employee m ON e.manager_id = m.employee_id 
)
SELECT * 
FROM EmployeeWithManagerCTE;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,employee_name,manager_name
0,Alice,Charlie
1,Bob,Charlie
2,Charlie,David
3,David,None


### Calculate the number of levels in the employee hierarchy for each employee.

In [24]:
cur.execute('''
WITH RECURSIVE EmployeeHierarchy AS (
    SELECT employee_ID, employee_name, manager_id, 1 AS level
    FROM Employee
    WHERE manager_id IS NULL

    UNION ALL

    SELECT e.employee_ID, e.employee_name, e.manager_id, eh.level + 1
    FROM Employee e
    JOIN EmployeeHierarchy eh ON e.manager_id = eh.employee_ID
)

SELECT employee_ID, employee_name, level
FROM EmployeeHierarchy;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,employee_ID,employee_name,level
0,4,David,1
1,3,Charlie,2
2,1,Alice,3
3,2,Bob,3


# Question 9: In an inventory database, identify all products that have never been ordered. Write an SQL query based on pairwise and non-pairwise to find the product details.
Hint:- To identify all products that have never been ordered in an inventory database, you can use both a pairwise comparison (with a subquery) and a non-pairwise approach (with a LEFT JOIN). 

In [25]:
cnn = sqlite3.connect('database5.db')
cur = cnn.cursor()

In [26]:
cur.execute('''
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    price DECIMAL(10, 2)
);
''')

In [2]:
cur.execute('''
INSERT INTO products (product_id, product_name, price) VALUES
(1, 'Product A', 10.99),
(2, 'Product B', 19.99),
(3, 'Product C', 15.49),
(4, 'Product D', 8.75);
''')

NameError: name 'cur' is not defined

In [28]:
cur.execute('''
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    product_id INT,
    quantity INT,
    order_date DATE,
    sales DECIMAL(10, 2),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
''')

In [29]:
cur.execute('''
INSERT INTO orders (order_id, product_id, quantity, order_date, sales) VALUES
(1, 1, 5, '2023-01-01', 54.95),
(2, 2, 3, '2023-01-02', 59.97),
(3, 1, 2, '2023-01-03', 21.98),
(4, 3, 1, '2023-01-04', 15.49);
''')

In [30]:
cur.execute('''
SELECT *
FROM products;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

cur.execute('''
SELECT *
FROM orders;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,product_id,product_name,price
0,1,Product A,10.99
1,2,Product B,19.99
2,3,Product C,15.49
3,4,Product D,8.75


,order_id,product_id,quantity,order_date,sales
0,1,1,5,2023-01-01,54.95
1,2,2,3,2023-01-02,59.97
2,3,1,2,2023-01-03,21.98
3,4,3,1,2023-01-04,15.49


### Identify the products that have never been ordered 
### Pairwise Comparison

In [31]:
cur.execute('''
SELECT *
FROM products
WHERE (product_id, product_name, price) NOT IN (
    SELECT p.product_id, p.product_name, p.price
    FROM products p
    JOIN orders o ON p.product_id = o.product_id
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,product_id,product_name,price
0,4,Product D,8.75


### Non-Pairwise Comparison

In [32]:
cur.execute('''
SELECT *
FROM products
WHERE product_id NOT IN (
    SELECT p.product_id
    FROM products p
    JOIN orders o ON p.product_id = o.product_id
)
AND product_name NOT IN (
    SELECT p.product_name
    FROM products p
    JOIN orders o ON p.product_id = o.product_id
)
AND price NOT IN (
    SELECT p.price
    FROM products p
    JOIN orders o ON p.product_id = o.product_id
)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,product_id,product_name,price
0,4,Product D,8.75


# Question 10: In a student records database, find all students who have a GPA greater than the average GPA of students in their department. Write an SQL query based on scalar, inline, pairwise, non-pairwise, multi-row, single row and correlated sub queries type to extract this information.
Hint :- You can find all students who have a GPA greater than the average GPA of students in their department using scalar, inline, and correlated sub queries. Here's how you can write SQL queries for each of these sub query types:

In [33]:
cnn = sqlite3.connect('database6.db')
cur = cnn.cursor()

In [34]:
cur.execute('''
CREATE TABLE student_records (
  student_id INT PRIMARY KEY,
  student_name VARCHAR(50),
  GPA DECIMAL(3, 2),
  department_id INT
);
''')

In [35]:
cur.execute('''
INSERT INTO student_records (student_id, student_name, GPA, department_id)
VALUES
  (1, 'John Doe', 3.5, 101),
  (2, 'Jane Smith', 3.8, 102),
  (3, 'Bob Johnson', 3.2, 101),
  (4, 'Alice Williams', 3.9, 103),
  (5, 'Charlie Brown', 3.6, 102),
  (6, 'Eva Martinez', 3.1, 101);
''')

In [36]:
cur.execute('''
SELECT *
FROM student_records;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)


,student_id,student_name,GPA,department_id
0,1,John Doe,3.5,101
1,2,Jane Smith,3.8,102
2,3,Bob Johnson,3.2,101
3,4,Alice Williams,3.9,103
4,5,Charlie Brown,3.6,102
5,6,Eva Martinez,3.1,101


### Find all students who have a GPA greater than the average GPA of all the students.

In [37]:
# Scalar Sub-Query
cur.execute('''
SELECT * 
FROM student_records
WHERE GPA > (
    SELECT AVG(GPA)
    FROM student_records
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,student_id,student_name,GPA,department_id
0,2,Jane Smith,3.8,102
1,4,Alice Williams,3.9,103
2,5,Charlie Brown,3.6,102


### Find all students who have a GPA greater than the average GPA of students in their department.

In [38]:
# Inline Sub-Query
cur.execute('''
SELECT sr.student_id, sr.student_name, sr.GPA, sr.department_id, avg_table.department_avg
FROM student_records sr
JOIN (
    SELECT department_id, AVG(GPA) AS department_avg
    FROM student_records
    GROUP BY department_id
) avg_table ON sr.department_id = avg_table.department_id
WHERE sr.GPA > avg_table.department_avg
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,student_id,student_name,GPA,department_id,department_avg
0,1,John Doe,3.5,101,3.266667
1,2,Jane Smith,3.8,102,3.700000


In [39]:
# Corelated Sub-Query
cur.execute('''
SELECT *
FROM student_records s1
WHERE GPA > (
    SELECT AVG(GPA) 
    FROM student_records s2
    WHERE s2.department_id = s1.department_id
)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

# To display department average
cur.execute('''
SELECT s1.*, (
    SELECT AVG(GPA) 
    FROM student_records s2
    WHERE s2.department_id = s1.department_id
) AS department_avg
FROM student_records s1
WHERE GPA > (
    SELECT AVG(GPA) 
    FROM student_records s2
    WHERE s2.department_id = s1.department_id
)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,student_id,student_name,GPA,department_id
0,1,John Doe,3.5,101
1,2,Jane Smith,3.8,102


,student_id,student_name,GPA,department_id,department_avg
0,1,John Doe,3.5,101,3.266667
1,2,Jane Smith,3.8,102,3.700000


# Question 11:In a company database, you want to find all employees who have the same job title as the employee with EmployeeID 101. Write an SQL query based on pair wise , non-pairwise, inline, corelated,multi-row,scalar & single row to accomplish this.
Hint: To find all employees who have the same job title as the employee with EmployeeID 101, you can use various SQL query techniques. Here are SQL queries based on pairwise, non-pairwise, inline, correlated, multi-row, and scalar subqueries:


In [40]:
cnn = sqlite3.connect('database7.db')
cur = cnn.cursor()

In [41]:
cur.execute('''
CREATE TABLE employees (
    EmployeeID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    JobTitle VARCHAR(50)
);
''')

In [42]:
cur.execute('''
INSERT INTO employees (EmployeeID, FirstName, LastName, JobTitle) VALUES
(101, 'John', 'Doe', 'Manager'),
(102, 'Jane', 'Smith', 'Developer'),
(103, 'Bob', 'Johnson', 'Manager'),
(104, 'Alice', 'Williams', 'Analyst'),
(105, 'Charlie', 'Brown', 'Developer');
''')

In [43]:
cur.execute('''
SELECT *
FROM employees;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,EmployeeID,FirstName,LastName,JobTitle
0,101,John,Doe,Manager
1,102,Jane,Smith,Developer
2,103,Bob,Johnson,Manager
3,104,Alice,Williams,Analyst
4,105,Charlie,Brown,Developer


### Find all employees who have the same job title as the employee with EmployeeID 101.

In [44]:
# Scalar Sub-Query
cur.execute('''
SELECT *
FROM employees
WHERE JobTitle = (
    SELECT JobTitle 
    FROM employees 
    WHERE EmployeeID = 101
)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,EmployeeID,FirstName,LastName,JobTitle
0,101,John,Doe,Manager
1,103,Bob,Johnson,Manager


In [45]:
# Multiple Row Sub-Query
cur.execute('''
SELECT *
FROM employees
WHERE JobTitle IN (
    SELECT JobTitle 
    FROM employees 
    WHERE EmployeeID = 101
)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,EmployeeID,FirstName,LastName,JobTitle
0,101,John,Doe,Manager
1,103,Bob,Johnson,Manager


# Question 12: In a sales database, you need to find the total sales amount for each product category and list only those categories where the total sales amount is greater than the average total sales amount for all categories. Write an SQL query for this by using single row, multi-row, scalar, inline, correlated, pairwise and non-pairwise.

Hint: To find the total sales amount for each product category and list only those categories where the total sales amount is greater than the average total sales amount for all categories, you can use different SQL query techniques. Here are SQL queries based on single row, multi-row, scalar, inline, correlated, pairwise, and non-pairwise subqueries:


In [46]:
cnn = sqlite3.connect('database8.db')
cur = cnn.cursor()

In [47]:
cur.execute('''
CREATE TABLE Sales (
    SaleID INT PRIMARY KEY,
    ProductID INT,
    Category VARCHAR(50),
    SalesAmount DECIMAL(10, 2),
    SaleDate DATE
);
''')

In [48]:
cur.execute('''
INSERT INTO Sales (SaleID, ProductID, Category, SalesAmount, SaleDate) VALUES
(1, 101, 'Electronics', 1500.50, '2023-01-01'),
(2, 102, 'Clothing', 500.75, '2023-01-02'),
(3, 103, 'Electronics', 1200.25, '2023-01-03'),
(4, 104, 'Clothing', 800.00, '2023-01-04'),
(5, 105, 'Electronics', 2000.80, '2023-01-05'),
(6, 106, 'Clothing', 300.50, '2023-01-06'),
(7, 107, 'Electronics', 1800.30, '2023-01-07'),
(8, 108, 'Clothing', 700.90, '2023-01-08'),
(9, 109, 'Electronics', 1600.60, '2023-01-09'),
(10, 110, 'Clothing', 400.25, '2023-01-10');
''')

In [49]:
cur.execute('''
SELECT *
FROM Sales;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,SaleID,ProductID,Category,SalesAmount,SaleDate
0,1,101,Electronics,1500.50,2023-01-01
1,2,102,Clothing,500.75,2023-01-02
2,3,103,Electronics,1200.25,2023-01-03
3,4,104,Clothing,800.00,2023-01-04
4,5,105,Electronics,2000.80,2023-01-05
5,6,106,Clothing,300.50,2023-01-06
6,7,107,Electronics,1800.30,2023-01-07
7,8,108,Clothing,700.90,2023-01-08
8,9,109,Electronics,1600.60,2023-01-09
9,10,110,Clothing,400.25,2023-01-10


### Find the total sales amount for each product category and list only those categories where the total sales amount is greater than the average total sales amount for all categories.

In [50]:
# Inline View Sub-Query
cur.execute('''
SELECT *
FROM (
    SELECT Category, SUM(SalesAmount) AS TotalSales
    FROM Sales
    GROUP BY Category
) AS CategoryTotalSales
WHERE TotalSales > (SELECT AVG(SalesAmount) FROM Sales)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,Category,TotalSales
0,Clothing,2702.40
1,Electronics,8102.45


In [51]:
# Multiple Row Sub-Query
cur.execute('''
SELECT Category, SUM(SalesAmount) AS TotalSales
FROM Sales
WHERE Category IN (
    SELECT DISTINCT Category
    FROM Sales
    WHERE (
        SELECT SUM(SalesAmount)
        FROM Sales
        GROUP BY Category
        HAVING SUM(SalesAmount) > (
            SELECT AVG(TotalSales)
            FROM (
                SELECT SUM(SalesAmount) AS TotalSales
                FROM Sales
                GROUP BY Category
            ) AS CategorySales
        )
    )
)
GROUP BY Category;
''')
result = cur.fetchall()
df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,Category,TotalSales
0,Clothing,2702.40
1,Electronics,8102.45


# Question 13: You have a library database and want to find all books that are currently checked out. Write an SQL query pairwise, non-pairwise, inline, correlated, multi-row, and scalar subqueries to retrieve the book titles.
Hint: To find all books that are currently checked out in a library database, you can use different SQL query techniques, including pairwise, non-pairwise, inline, correlated, multi-row, and scalar subqueries. Here are SQL queries for each of these subquery types:

In [52]:
cnn = sqlite3.connect('database9.db')
cur = cnn.cursor()

In [53]:
cur.execute('''
CREATE TABLE Books (
    BookID INT PRIMARY KEY,
    Title VARCHAR(255) NOT NULL,
    Author VARCHAR(255),
    ISBN VARCHAR(20)
);
''')

In [54]:
cur.execute('''
CREATE TABLE Checkouts (
    CheckoutID INT PRIMARY KEY,
    BookID INT,
    CheckoutDate DATE,
    ReturnDate DATE,
    FOREIGN KEY (BookID) REFERENCES Books(BookID)
);
''')

In [55]:
cur.execute('''
INSERT INTO Books (BookID, Title, Author, ISBN)
VALUES
    (1, 'The Great Gatsby', 'F. Scott Fitzgerald', '978-3-16-148410-0'),
    (2, 'To Kill a Mockingbird', 'Harper Lee', '978-0-06-112008-4'),
    (3, '1984', 'George Orwell', '978-0-45-152493-5'),
    (4, 'Pride and Prejudice', 'Jane Austen', '978-1-41-650305-8'),
    (5, 'The Catcher in the Rye', 'J.D. Salinger', '978-0-31-676948-0'),
    (6, 'The Hobbit', 'J.R.R. Tolkien', '978-0-26-110334-4'),
    (7, 'The Da Vinci Code', 'Dan Brown', '978-0-76-117490-4'),
    (8, 'The Shining', 'Stephen King', '978-0-30-774365-7'),
    (9, 'Brave New World', 'Aldous Huxley', '978-0-06-085052-4'),
    (10, 'The Lord of the Rings', 'J.R.R. Tolkien', '978-0-54-400341-5');
''')

In [56]:
cur.execute('''
INSERT INTO Checkouts (CheckoutID, BookID, CheckoutDate, ReturnDate)
VALUES
    (1, 1, '2023-01-01', '2023-01-15'),
    (2, 2, '2023-02-01', NULL),
    (3, 3, '2023-03-01', '2023-03-15'),
    (4, 4, '2023-04-01', '2023-04-15'),
    (5, 5, '2023-05-01', '2023-05-15'),
    (6, 6, '2023-06-01', NULL),
    (7, 7, '2023-07-01', '2023-07-15'),
    (8, 8, '2023-08-01', '2023-08-15'),
    (9, 9, '2023-09-01', '2023-09-15'),
    (10, 10, '2023-10-01', NULL);
''')

### Find all books that are currently checked out.

In [57]:
cur.execute('''
SELECT Books.BookID, Books.Title, Checkouts.ReturnDate
FROM Books
LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
WHERE Checkouts.ReturnDate IS NULL;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,BookID,Title,ReturnDate
0,2,To Kill a Mockingbird,None
1,6,The Hobbit,None
2,10,The Lord of the Rings,None


In [58]:
# Inline View Sub-Query
cur.execute('''
SELECT *
FROM (
    SELECT Books.BookID, Books.Title, Checkouts.ReturnDate
    FROM Books
    LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
    WHERE Checkouts.ReturnDate IS NULL
)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,BookID,Title,ReturnDate
0,2,To Kill a Mockingbird,None
1,6,The Hobbit,None
2,10,The Lord of the Rings,None


In [59]:
# Multiple Row Sub-Query
cur.execute('''
SELECT Books.BookID, Books.Title, Checkouts.ReturnDate
FROM Books
LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
WHERE Books.BookID IN (
    SELECT Books.BookID
    FROM Books
    LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
    WHERE Checkouts.ReturnDate IS NULL
)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,BookID,Title,ReturnDate
0,2,To Kill a Mockingbird,None
1,6,The Hobbit,None
2,10,The Lord of the Rings,None


In [60]:
# Pairwise Comparison Sub-Query
cur.execute('''
SELECT Books.BookID, Books.Title, Checkouts.ReturnDate
FROM Books
LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
WHERE (Books.BookID, Books.Title) IN (
    SELECT Books.BookID, Books.Title
    FROM Books
    LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
    WHERE Checkouts.ReturnDate IS NULL
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,BookID,Title,ReturnDate
0,2,To Kill a Mockingbird,None
1,6,The Hobbit,None
2,10,The Lord of the Rings,None


In [61]:
# Non-Pairwise Comparison Sub-Query
cur.execute('''
SELECT Books.BookID, Books.Title, Checkouts.ReturnDate
FROM Books
LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
WHERE Books.BookID IN (
    SELECT Books.BookID
    FROM Books
    LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
    WHERE Checkouts.ReturnDate IS NULL
) AND  Books.Title IN (
    SELECT Books.Title
    FROM Books
    LEFT JOIN Checkouts ON Books.BookID = Checkouts.BookID
    WHERE Checkouts.ReturnDate IS NULL
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,BookID,Title,ReturnDate
0,2,To Kill a Mockingbird,None
1,6,The Hobbit,None
2,10,The Lord of the Rings,None


# Question 14: You are given a list of numbers, and you need to find the sum of all even numbers in the list using Python's built-in functions. Write a Python function to accomplish this task.
### numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [62]:
def sum_of_even_numbers(numbers):
    return sum(num for num in numbers if num % 2 == 0)

# Example usage
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = sum_of_even_numbers(numbers)
print(result)

30


# Question 15: Create the given SQL dataset, insert data and perform the FUNCTION on questions:-

Suppose you have a dataset named 'sales' with the following columns:

1. order_id (INT)
2. order_date (DATE)
3. delivery_date (DATETIME)
4. product_name (VARCHAR)
5. quantity (INT)
6. unit_price (DECIMAL)

### Questions:
1. How many orders were placed in the month of May 2023?
2. What was the total revenue generated in the first quarter (January to March) of 2023?
3. Calculate the average delivery time in hours for all orders.
4. Which product had the highest total sales quantity in 2023?

In [63]:
cnn = sqlite3.connect('database10.db')
cur = cnn.cursor()

In [64]:
cur.execute('''
CREATE TABLE sales (
    order_id INT,
    order_date DATE,
    delivery_date DATETIME,
    product_name VARCHAR(255),
    quantity INT,
    unit_price DECIMAL(10, 2)
);
''')

In [65]:
cur.execute('''
INSERT INTO sales VALUES
(1, '2023-05-01', '2023-05-02 12:30:00', 'Product A', 10, 20.00),
(2, '2023-05-03', '2023-05-05 14:45:00', 'Product B', 8, 25.00),
(3, '2023-03-15', '2023-03-18 09:00:00', 'Product A', 5, 22.50),
(4, '2023-01-10', '2023-01-12 17:30:00', 'Product C', 12, 15.00),
(5, '2023-02-22', '2023-02-25 08:15:00', 'Product B', 15, 18.00),
(6, '2023-06-05', '2023-06-08 10:00:00', 'Product C', 7, 30.00);
''')

In [66]:
cur.execute('''
SELECT *
FROM sales
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,order_id,order_date,delivery_date,product_name,quantity,unit_price
0,1,2023-05-01,2023-05-02 12:30:00,Product A,10,20.0
1,2,2023-05-03,2023-05-05 14:45:00,Product B,8,25.0
2,3,2023-03-15,2023-03-18 09:00:00,Product A,5,22.5
3,4,2023-01-10,2023-01-12 17:30:00,Product C,12,15.0
4,5,2023-02-22,2023-02-25 08:15:00,Product B,15,18.0
5,6,2023-06-05,2023-06-08 10:00:00,Product C,7,30.0


##### How many orders were placed in the month of May 2023?

In [67]:
cur.execute('''
SELECT COUNT(*) AS total_orders
FROM sales
WHERE strftime('%Y-%m', order_date) = '2023-05';
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,total_orders
0,2


##### What was the total revenue generated in the first quarter (January to March) of 2023?

In [68]:
cur.execute('''
SELECT SUM(quantity * unit_price) FROM sales
WHERE order_date BETWEEN '2023-01-01' AND '2023-03-31';
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,SUM(quantity * unit_price)
0,562.5


##### Calculate the average delivery time in hours for all orders.

In [69]:
cur.execute('''
SELECT AVG(strftime('%s', delivery_date) - strftime('%s', order_date)) / 3600 AS avg_delivery_time_hours
FROM sales;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,avg_delivery_time_hours
0,68.0


##### Which product had the highest total sales quantity in 2023?

In [70]:
cur.execute('''
SELECT product_name, SUM(quantity) AS total_sales_quantity
FROM sales
WHERE strftime('%Y', order_date) = '2023'
GROUP BY product_name
ORDER BY total_sales_quantity DESC
LIMIT 1;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,product_name,total_sales_quantity
0,Product B,23


# Question 16:

In [71]:
cnn = sqlite3.connect('database11.db')
cur = cnn.cursor()

In [72]:
cur.execute('''
CREATE TABLE orders (
    order_id INT,
    customer_id INT,
    order_date DATETIME,
    total_amount DECIMAL(10, 2)
);
''')

In [73]:
cur.execute('''
INSERT INTO orders VALUES
(1, 101, '2023-01-15 08:30:00', 150.00),
(2, 102, '2023-02-20 14:45:00', 200.00),
(3, 103, '2023-03-05 10:20:00', 75.00),
(4, 101, '2023-03-18 11:10:00', 100.00);
''')

In [74]:
cur.execute('''
SELECT *
FROM orders
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,order_id,customer_id,order_date,total_amount
0,1,101,2023-01-15 08:30:00,150
1,2,102,2023-02-20 14:45:00,200
2,3,103,2023-03-05 10:20:00,75
3,4,101,2023-03-18 11:10:00,100


##### How many orders were placed in the month of February 2023?

In [75]:
cur.execute('''
SELECT COUNT(*) 
FROM orders 
WHERE strftime('%Y-%m', order_date) = '2023-02'
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,COUNT(*)
0,1


##### What is the average total amount of orders placed in March 2023?


In [76]:
cur.execute('''
SELECT AVG(total_amount) 
FROM orders 
WHERE strftime('%Y-%m', order_date) = '2023-03'
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,AVG(total_amount)
0,87.5


##### How many days elapsed between the first and last order placed in the dataset?

In [77]:
cur.execute('''
SELECT 
MIN(order_date) AS min_date, 
MAX(order_date) AS max_date FROM orders
''')

# Fetch the result
result = cur.fetchone()

# Get the minimum and maximum dates
min_date, max_date = result

# Calculate the number of days elapsed
min_date = dt.strptime(min_date, "%Y-%m-%d %H:%M:%S")
max_date = dt.strptime(max_date, "%Y-%m-%d %H:%M:%S")
days_elapsed = (max_date - min_date).days

# Print the result
print(f"Number of days elapsed between the first and last order: {days_elapsed} days")

Number of days elapsed between the first and last order: 62 days


##### What is the total sales made on weekends (Saturday and Sunday) in the dataset?


In [78]:
cur.execute('''
SELECT SUM(total_amount) 
FROM orders 
WHERE strftime('%w', order_date) IN ('0', '6')
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,SUM(total_amount)
0,325


# Question 17: Why ER Model is used in DBMS? Draw the ER Model for Student information with respective attributes

#### The Entity-Relationship (ER) model is used in Database Management Systems (DBMS) for several important reasons:

1. Conceptual Clarity:

The ER model provides a high-level, abstract representation of the data in a system. It uses entities to represent real-world objects, attributes to represent properties of those objects, and relationships to represent connections between entities. This abstraction helps in understanding the structure of the data without getting into the details of how the data is physically stored.

2. Communication:

ER diagrams offer a visual means of communication between different stakeholders involved in the development of a database system. The graphical representation is easily understandable by both technical and non-technical individuals, fostering effective communication and collaboration.

3. Database Design:

ER modeling is an essential step in the process of designing a database. It helps in identifying and defining the entities, attributes, and relationships that need to be represented in the database. This structured approach guides the design process and ensures that the resulting database is well-organized and efficient.

4. Normalization:

The ER model facilitates the normalization process in database design. Normalization is the process of organizing data in a database to reduce redundancy and improve data integrity. ER diagrams help in identifying relationships between entities and attributes, which aids in the normalization process.

5. Database Maintenance and Modification:

ER diagrams serve as a valuable reference for maintaining and modifying a database. When changes or updates are required, the ER model provides a visual representation of the existing structure, making it easier to understand how modifications might impact the overall system.

6. Documentation:

ER diagrams act as documentation for the database schema. They provide a comprehensive view of the entities, attributes, and relationships, making it easier for developers, administrators, and other stakeholders to understand the database structure.
D
7. atabase Optimization:

During the database design phase, the ER model can help identify potential areas for optimization. By visualizing the relationships between entities, designers can make informed decisions about indexing, partitioning, and other optimization strategies.


#### let's create an ER diagram for the "Student Information" scenario:

#### Entities and Attributes:

- Strong Entity:
Entity: Student
Attributes: StudentID (Primary Key), Name, DateOfBirth, Address, etc.

- Weak Entity:
Entity: CourseRegistration
Attributes: RegistrationID (Partial Key), CourseID (Partial Key), Semester, etc.

#### Relationships:

- One-to-One (1-1) Relationship:
Relationship: Student has StudentID (1-1) StudentID in CourseRegistration

- One-to-Many (1-M) Relationship:
Relationship: Student takes (1-M) CourseRegistration

- Many-to-Many (M-M) Relationship:
Relationship: Student takes (M-M) Course

# Question 18:

In [79]:
cnn = sqlite3.connect('database12.db')
cur = cnn.cursor()

In [80]:
cur.execute('''
CREATE TABLE emp_details (
    e_id INT PRIMARY KEY,
    e_name VARCHAR(255),
    e_address VARCHAR(255),
    e_age INT,
    phone_no VARCHAR(15)
);
''')

In [81]:
cur.execute('''
INSERT INTO emp_details VALUES
(1, 'John Doe', '123 Main St', 30, '555-1234'),
(2, 'Jane Smith', '456 Oak St', 25, '555-5678'),
(3, 'Bob Johnson', '789 Pine St', 35, '555-9876'),
(4, 'Alice Brown', '321 Cedar St', 28, '555-4321'),
(5, 'Charlie Wilson', '555 Elm St', 40, '555-6789'),
(6, 'Eva Davis', '999 Maple St', 22, '555-3456');
''')

In [82]:
cur.execute('''
SELECT *
FROM emp_details
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,e_id,e_name,e_address,e_age,phone_no
0,1,John Doe,123 Main St,30,555-1234
1,2,Jane Smith,456 Oak St,25,555-5678
2,3,Bob Johnson,789 Pine St,35,555-9876
3,4,Alice Brown,321 Cedar St,28,555-4321
4,5,Charlie Wilson,555 Elm St,40,555-6789
5,6,Eva Davis,999 Maple St,22,555-3456


# Question 19:

In [83]:
cnn = sqlite3.connect('database13.db')
cur = cnn.cursor()

In [84]:
cur.execute('''
CREATE TABLE sales_data (
  state VARCHAR(255) NOT NULL,
  department VARCHAR(255) NOT NULL,
  sales_person VARCHAR(255) NOT NULL
);
''')

In [85]:
cur.execute('''
INSERT INTO sales_data (state, department, sales_person)
VALUES
  ('Texas', 'Clothing', 'Jake'),
  ('Florida', 'Home Décor', 'Amy'),
  ('Texas', 'Jewellery', 'Rachel'),
  ('Texas', 'Home Décor', 'Tony'),
  ('Florida', 'Clothing', 'Micky'),
  ('Texas', 'Clothing', 'Howard'),
  ('Florida', 'Jewellery', 'Tracey');
''')

In [86]:
cur.execute('''
SELECT *
FROM sales_data
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,state,department,sales_person
0,Texas,Clothing,Jake
1,Florida,Home Décor,Amy
2,Texas,Jewellery,Rachel
3,Texas,Home Décor,Tony
4,Florida,Clothing,Micky
5,Texas,Clothing,Howard
6,Florida,Jewellery,Tracey


##### Write sub query to display highest state from the above table.


In [87]:
cur.execute('''
SELECT DISTINCT state
FROM sales_data
WHERE LENGTH(state) = (
  SELECT MAX(LENGTH(state))
  FROM sales_data
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,state
0,Florida


##### Write sub query to display the name of the sales person  who work in Clothing departments.

In [88]:
# Multiple Row Sub-Query
cur.execute('''
SELECT sales_person, department
FROM sales_data
WHERE sales_person IN (
    SELECT sales_person
    FROM sales_data
    WHERE department = 'Clothing'
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,sales_person,department
0,Jake,Clothing
1,Micky,Clothing
2,Howard,Clothing


In [89]:
# Inline View Sub-Query
cur.execute('''
SELECT *
FROM (
    SELECT sales_person, department
    FROM sales_data
    WHERE department = 'Clothing'
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,sales_person,department
0,Jake,Clothing
1,Micky,Clothing
2,Howard,Clothing


##### Find out the count of states from the above table:

In [90]:
# Inline View Sub-Query
cur.execute('''
SELECT *
FROM (
    SELECT COUNT(DISTINCT state) as state_count
    FROM sales_data);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,state_count
0,2


# Question 20

In [91]:
cnn = sqlite3.connect('database14.db')
cur = cnn.cursor()

In [92]:
cur.execute('''
CREATE TABLE Employee (
    Emp_id CHAR(4),
    Emp_name VARCHAR(50),
    Emp_salary INT,
    Emp_add VARCHAR(50),
    Emp_mobno BIGINT
);
''')

In [93]:
cur.execute('''
INSERT INTO Employee (Emp_id, Emp_name, Emp_salary, Emp_add, Emp_mobno)
VALUES
('0101', 'Rahul', 10000, 'Delhi', 9005689758),
('0102', 'Ajit', 20000, 'Mumbai', 9091659878),
('0103', 'Nupur', 30000, 'MP', 8025947895),
('0104', 'Swati', 40000, 'UP', 9958974635);
''')

In [94]:
cur.execute('''
SELECT *
FROM Employee
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,Emp_id,Emp_name,Emp_salary,Emp_add,Emp_mobno
0,0101,Rahul,10000,Delhi,9005689758
1,0102,Ajit,20000,Mumbai,9091659878
2,0103,Nupur,30000,MP,8025947895
3,0104,Swati,40000,UP,9958974635


##### Write subquery to display the highest salary from the above table:



In [95]:
# Scalar Sub-Query
cur.execute('''
SELECT Emp_salary
FROM Employee
WHERE Emp_salary IN (
    SELECT MAX(Emp_salary) 
    FROM Employee
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,Emp_salary
0,40000


##### Write subquery to display the name of the employee having the maximum salary:



In [96]:
# Scalar Sub-Query
cur.execute('''
SELECT Emp_name
FROM Employee
WHERE Emp_salary = (
    SELECT MAX(Emp_salary) 
    FROM Employee
);
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,Emp_name
0,Swati


##### Find out the average salary from the above table:

In [97]:
cur.execute('''
SELECT * 
FROM (
    SELECT AVG(Emp_salary) 
    FROM Employee
)
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,AVG(Emp_salary)
0,25000.0


# Question 24:

In [98]:
# Import the required library for connecting to MySQL
!pip install mysql-connector-python
import mysql.connector as sql

In [99]:
# Establish a connection to the MySQL server with provided credentials
mydb = sql.connect(
    host="localhost",       
    user="root",            
    password="11111111" 
)

In [100]:
# Create a cursor object to interact with the MySQL database
mycursor = mydb.cursor()

In [101]:
# Create a new database
mycursor.execute('''
CREATE DATABASE MyStore;
''')

In [102]:
# Switch to the database
mycursor.execute('''
USE MyStore;
''')

In [103]:
# Create a new table
mycursor.execute('''
CREATE TABLE ProductType (
  ProductTypeID int PRIMARY KEY,
  ProductTypeName varchar(255) NOT NULL
);
''')

In [104]:
mycursor.execute('''
INSERT INTO ProductType VALUES 
(1, 'Electronics'),
(2, 'Clothing');
''')

In [105]:
mycursor.execute('''
CREATE TABLE Product (
  ProductID int PRIMARY KEY,
  ProductTypeID int NOT NULL,
  ProductName varchar(255) NOT NULL,
  UnitSalePrice decimal(10,2) NOT NULL,
  UnitCosts decimal(10,2) NOT NULL,
  Volume int NOT NULL,
  VariableCosts decimal(10,2) NOT NULL,
  IndirectCosts decimal(10,2) NOT NULL,
  FOREIGN KEY (ProductTypeID) REFERENCES ProductType(ProductTypeID)
);
''')

In [106]:
mycursor.execute('''
INSERT INTO Product VALUES 
(1, 1, 'Product A', 100.00, 50.00, 1000, 20.00, 10.00),
(2, 2, 'Product B', 50.00, 25.00, 500, 15.00, 5.00),
(3, 1, 'Product C', 75.00, 37.50, 750, 18.75, 6.25);
''')

In [107]:
# Create a new table
mycursor.execute('''
CREATE TABLE Time (
  TimeID int PRIMARY KEY,
  Year int NOT NULL,
  Quarter int NOT NULL,
  Month int NOT NULL
);
''')

In [108]:
mycursor.execute('''
INSERT INTO Time VALUES 
(1, 2023, 1, 1),
(2, 2023, 1, 2),
(3, 2023, 1, 3);
''')

In [109]:
# Create a new table
mycursor.execute('''
CREATE TABLE FinancialFacts (
  FinancialFactsID int PRIMARY KEY,
  ProductID int NOT NULL,
  TimeID int NOT NULL,
  Profit decimal(10,2) NOT NULL,
  Revenues decimal(10,2) NOT NULL,
  Costs decimal(10,2) NOT NULL,
  FOREIGN KEY (ProductID) REFERENCES Product(ProductID),
  FOREIGN KEY (TimeID) REFERENCES Time(TimeID)
);
''')

In [110]:
mycursor.execute('''
INSERT INTO FinancialFacts VALUES 
(1, 1, 1, 50.00, 100.00, 50.00),
(2, 2, 2, 25.00, 50.00, 25.00),
(3, 3, 3, 37.50, 75.00, 37.50);
''')

In [111]:
# Create a new table
mycursor.execute('''
CREATE TABLE City (
  CityID int PRIMARY KEY,
  CityName varchar(255) NOT NULL
);
''')

In [112]:
mycursor.execute('''
INSERT INTO City VALUES 
(1, 'City A'),
(2, 'City B');
''')

In [113]:
# Create a new table
mycursor.execute('''
CREATE TABLE Country (
  CountryID int PRIMARY KEY,
  CountryName varchar(255) NOT NULL
);
''')

In [114]:
mycursor.execute('''
INSERT INTO Country
VALUES (1, 'Country 1');
''')

In [115]:
# Create a new table
mycursor.execute('''
CREATE TABLE Location (
  LocationID int PRIMARY KEY,
  Name varchar(255) NOT NULL,
  Position int NOT NULL,
  CityID int NOT NULL,
  CountryID int NOT NULL,
  FOREIGN KEY (CityID) REFERENCES City(CityID),
  FOREIGN KEY (CountryID) REFERENCES Country(CountryID)
);
''')

In [116]:
mycursor.execute('''
INSERT INTO Location VALUES 
(1, 'Store 1', 1, 1, 1),
(2, 'Store 2', 2, 1, 1),
(3, 'Warehouse 1', 3, 2, 1);
''')

In [117]:
# Create a new table
mycursor.execute('''
CREATE TABLE VendorDivision (
  VendorDivisionID int PRIMARY KEY,
  VendorDivisionName varchar(255) NOT NULL
);
''')

In [118]:
mycursor.execute('''
INSERT INTO VendorDivision VALUES 
(1, 'Division A'),
(2, 'Division B');
''')

In [119]:
# Create a new table
mycursor.execute('''
CREATE TABLE Vendor (
  VendorID int PRIMARY KEY,
  VendorName varchar(255) NOT NULL,
  VendorDivisionID int NOT NULL,
  FOREIGN KEY (VendorDivisionID) REFERENCES VendorDivision(VendorDivisionID)
);
''')

In [120]:
mycursor.execute('''
INSERT INTO Vendor VALUES 
(1, 'Vendor A', 1),
(2, 'Vendor B', 2);
''')

In [121]:
mycursor.execute('''
SELECT * 
FROM Product
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM ProductType
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM Time
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM FinancialFacts
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM Location
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM City
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM Country
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM Vendor
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM VendorDivision
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

,ProductID,ProductTypeID,ProductName,UnitSalePrice,UnitCosts,Volume,VariableCosts,IndirectCosts
0,1,1,Product A,100.00,50.00,1000,20.00,10.00
1,2,2,Product B,50.00,25.00,500,15.00,5.00
2,3,1,Product C,75.00,37.50,750,18.75,6.25


,ProductTypeID,ProductTypeName
0,1,Electronics
1,2,Clothing


,TimeID,Year,Quarter,Month
0,1,2023,1,1
1,2,2023,1,2
2,3,2023,1,3


,FinancialFactsID,ProductID,TimeID,Profit,Revenues,Costs
0,1,1,1,50.00,100.00,50.00
1,2,2,2,25.00,50.00,25.00
2,3,3,3,37.50,75.00,37.50


,LocationID,Name,Position,CityID,CountryID
0,1,Store 1,1,1,1
1,2,Store 2,2,1,1
2,3,Warehouse 1,3,2,1


,CityID,CityName
0,1,City A
1,2,City B


,CountryID,CountryName
0,1,Country 1


,VendorID,VendorName,VendorDivisionID
0,1,Vendor A,1
1,2,Vendor B,2


,VendorDivisionID,VendorDivisionName
0,1,Division A
1,2,Division B


In [122]:
# Close the MySQL connection
mydb.close()

# Question 25:

In [123]:
# Import the required library for connecting to MySQL
!pip install mysql-connector-python
import mysql.connector as sql

In [124]:
# Establish a connection to the MySQL server with provided credentials
mydb = sql.connect(
    host="localhost",       
    user="root",            
    password="11111111" 
)

In [125]:
# Create a cursor object to interact with the MySQL database
mycursor = mydb.cursor()

In [126]:
# Create a new database
mycursor.execute('''
CREATE DATABASE MyDB1;
''')

In [127]:
# Switch to the database
mycursor.execute('''
USE MyDB1;
''')

In [128]:
# Create a new table
mycursor.execute('''
CREATE TABLE Employees (
    EmployeeID int NOT NULL,
    FirstName varchar(255) NOT NULL,
    MiddleInitial varchar(255),
    LastName varchar(255) NOT NULL,
    PRIMARY KEY (EmployeeID)
);
''')

In [129]:
mycursor.execute('''
INSERT INTO Employees (EmployeeID, FirstName, MiddleInitial, LastName)
VALUES
    (1, 'John', 'D', 'Doe'),
    (2, 'Jane', 'M', 'Smith');
''')

In [130]:
# Create a new table
mycursor.execute('''
CREATE TABLE Customers (
    CustomerID int NOT NULL,
    FirstName varchar(255) NOT NULL,
    MiddleInitial varchar(255),
    LastName varchar(255) NOT NULL,
    PRIMARY KEY (CustomerID)
);
''')

In [131]:
mycursor.execute('''
INSERT INTO Customers (CustomerID, FirstName, MiddleInitial, LastName)
VALUES
    (1, 'Alice', 'A', 'Anderson'),
    (2, 'Bob', 'B', 'Brown');
''')

In [132]:
# Create a new table
mycursor.execute('''
CREATE TABLE Products (
    ProductID int NOT NULL,
    Name varchar(255) NOT NULL,
    Price decimal(10,2) NOT NULL,
    PRIMARY KEY (ProductID)
);
''')

In [133]:
mycursor.execute('''
INSERT INTO Products (ProductID, Name, Price)
VALUES
    (1, 'Product A', 100.00),
    (2, 'Product B', 50.00);
''')

In [134]:
# Create a new table
mycursor.execute('''
CREATE TABLE Sales (
    SalesID int NOT NULL,
    SalesPersonID int NOT NULL,
    CustomerID int NOT NULL,
    ProductID int NOT NULL,
    Quantity int NOT NULL,
    PRIMARY KEY (SalesID),
    FOREIGN KEY (SalesPersonID) REFERENCES Employees (EmployeeID),
    FOREIGN KEY (CustomerID) REFERENCES Customers (CustomerID),
    FOREIGN KEY (ProductID) REFERENCES Products (ProductID)
);
''')

In [135]:
mycursor.execute('''
INSERT INTO Sales (SalesID, SalesPersonID, CustomerID, ProductID, Quantity)
VALUES
    (1, 1, 1, 1, 5),
    (2, 2, 2, 2, 3);
''')

In [136]:
mycursor.execute('''
SELECT * 
FROM Employees
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM Customers
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM Products
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM Sales 
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

,EmployeeID,FirstName,MiddleInitial,LastName
0,1,John,D,Doe
1,2,Jane,M,Smith


,CustomerID,FirstName,MiddleInitial,LastName
0,1,Alice,A,Anderson
1,2,Bob,B,Brown


,ProductID,Name,Price
0,1,Product A,100.00
1,2,Product B,50.00


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity
0,1,1,1,1,5
1,2,2,2,2,3


In [137]:
# Close the MySQL connection
mydb.close()

# Question 26:

In [138]:
# Import the required library for connecting to MySQL
!pip install mysql-connector-python
import mysql.connector as sql

In [139]:
# Establish a connection to the MySQL server with provided credentials
mydb = sql.connect(
    host="localhost",       
    user="root",            
    password="11111111" 
)

In [140]:
# Create a cursor object to interact with the MySQL database
mycursor = mydb.cursor()

In [141]:
# Create a new database
mycursor.execute('''
CREATE DATABASE MyDB2;
''')

In [142]:
# Switch to the database
mycursor.execute('''
USE MyDB2;
''')

In [143]:
# Create a new table
mycursor.execute('''
CREATE TABLE orders (
  order_id INT PRIMARY KEY,
  order_date DATE,
  year INT,
  quarter INT,
  month INT
);
''')

In [144]:
mycursor.execute('''
INSERT INTO orders (order_id, order_date, year, quarter, month)
VALUES
  (1, '2023-01-15', 2023, 1, 1),
  (2, '2023-02-20', 2023, 1, 2),
  (3, '2023-03-25', 2023, 1, 3);
''')

In [145]:
# Create a new table
mycursor.execute('''
CREATE TABLE products (
  product_id INT PRIMARY KEY,
  product_name VARCHAR(255),
  product_category VARCHAR(255),
  unit_price DECIMAL(10,2)
);
''')

In [146]:
mycursor.execute('''
INSERT INTO products (product_id, product_name, product_category, unit_price)
VALUES
  (101, 'Product A', 'Category 1', 25.99),
  (102, 'Product B', 'Category 2', 15.50),
  (103, 'Product C', 'Category 1', 30.75);
''')

In [147]:
# Create a new table
mycursor.execute('''
CREATE TABLE customers (
  customer_id INT PRIMARY KEY,
  customer_name VARCHAR(255),
  address VARCHAR(255),
  zip VARCHAR(255)
);
''')

In [148]:
mycursor.execute('''
INSERT INTO customers (customer_id, customer_name, address, zip)
VALUES
  (201, 'Customer X', '123 Main St', '12345'),
  (202, 'Customer Y', '456 Oak St', '67890'),
  (203, 'Customer Z', '789 Pine St', '34567');
''')

In [149]:
# Create a new table
mycursor.execute('''
CREATE TABLE employers (
  employer_id INT PRIMARY KEY,
  employer_name VARCHAR(255),
  title VARCHAR(255),
  department VARCHAR(255),
  region VARCHAR(255)
);
''')

In [150]:
mycursor.execute('''
INSERT INTO employers (employer_id, employer_name, title, department, region)
VALUES
  (301, 'Employer 1', 'Manager', 'Sales', 'East'),
  (302, 'Employer 2', 'Supervisor', 'Marketing', 'West'),
  (303, 'Employer 3', 'Director', 'Finance', 'North');
''')

In [151]:
# Create a new table
mycursor.execute('''
CREATE TABLE sales (
  product_id INT,
  order_id INT,
  customer_id INT,
  employer_id INT,
  total DECIMAL(10,2),
  quantity INT,
  discount DECIMAL(10,2),
  FOREIGN KEY (product_id) REFERENCES products(product_id),
  FOREIGN KEY (order_id) REFERENCES orders(order_id),
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
  FOREIGN KEY (employer_id) REFERENCES employers(employer_id)
);


''')

In [152]:
mycursor.execute('''
INSERT INTO sales (product_id, order_id, customer_id, employer_id, total, quantity, discount)
VALUES
  (101, 1, 201, 301, 100.00, 2, 5.00),
  (102, 2, 202, 302, 75.50, 3, 2.50),
  (103, 3, 203, 303, 150.25, 1, 10.00);
''')

In [153]:
mycursor.execute('''
SELECT * 
FROM orders
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM products 
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM customers 
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM employers  
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM sales   
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

,order_id,order_date,year,quarter,month
0,1,2023-01-15,2023,1,1
1,2,2023-02-20,2023,1,2
2,3,2023-03-25,2023,1,3


,product_id,product_name,product_category,unit_price
0,101,Product A,Category 1,25.99
1,102,Product B,Category 2,15.50
2,103,Product C,Category 1,30.75


,customer_id,customer_name,address,zip
0,201,Customer X,123 Main St,12345
1,202,Customer Y,456 Oak St,67890
2,203,Customer Z,789 Pine St,34567


,employer_id,employer_name,title,department,region
0,301,Employer 1,Manager,Sales,East
1,302,Employer 2,Supervisor,Marketing,West
2,303,Employer 3,Director,Finance,North


,product_id,order_id,customer_id,employer_id,total,quantity,discount
0,101,1,201,301,100.00,2,5.00
1,102,2,202,302,75.50,3,2.50
2,103,3,203,303,150.25,1,10.00


In [154]:
# Close the MySQL connection
mydb.close()

# Question 27:

In [155]:
# Import the required library for connecting to MySQL
!pip install mysql-connector-python
import mysql.connector as sql

In [156]:
# Establish a connection to the MySQL server with provided credentials
mydb = sql.connect(
    host="localhost",       
    user="root",            
    password="11111111" 
)

In [157]:
# Create a cursor object to interact with the MySQL database
mycursor = mydb.cursor()

In [158]:
# Create a new database
mycursor.execute('''
CREATE DATABASE MyDB3;
''')

In [159]:
# Switch to the database
mycursor.execute('''
USE MyDB3;
''')

In [160]:
# Create a new table
mycursor.execute('''
CREATE TABLE DimDate (
  Date DATE PRIMARY KEY,
  Day VARCHAR(255),
  Month VARCHAR(255),
  Quarter VARCHAR(255),
  Year VARCHAR(255)
);
''')

In [161]:
mycursor.execute('''
INSERT INTO DimDate (Date, Day, Month, Quarter, Year)
VALUES
  ('2023-01-15', 'Monday', 'January', 'Q1', '2023'),
  ('2023-02-20', 'Wednesday', 'February', 'Q1', '2023'),
  ('2023-03-25', 'Saturday', 'March', 'Q1', '2023');
''')

In [162]:
# Create a new table
mycursor.execute('''
CREATE TABLE DimProduct (
  ProductId INT PRIMARY KEY,
  Title VARCHAR(255),
  Author VARCHAR(255),
  Category VARCHAR(255)
);
''')

In [163]:
mycursor.execute('''
INSERT INTO DimProduct (ProductId, Title, Author, Category)
VALUES
  (101, 'Book A', 'Author X', 'Fiction'),
  (102, 'Book B', 'Author Y', 'Non-Fiction'),
  (103, 'Book C', 'Author Z', 'Science');
''')

In [164]:
# Create a new table
mycursor.execute('''
CREATE TABLE DimCustomer (
  CustomerId INT PRIMARY KEY,
  CustomerName VARCHAR(255),
  CustomerType VARCHAR(255),
  City VARCHAR(255)
);
''')

In [165]:
mycursor.execute('''
INSERT INTO DimCustomer (CustomerId, CustomerName, CustomerType, City)
VALUES
  (201, 'Customer X', 'Regular', 'New York'),
  (202, 'Customer Y', 'VIP', 'Los Angeles'),
  (203, 'Customer Z', 'Regular', 'Chicago');
''')

In [166]:
# Create a new table
mycursor.execute('''
CREATE TABLE FactSales (
  OrderRef VARCHAR(255) PRIMARY KEY,
  CustomerId INT,
  ProductId INT,
  Quantity INT,
  SoldPrice DECIMAL(10,2),
  SalesAmount DECIMAL(10,2),
  SaleDate DATE,
  FOREIGN KEY (CustomerId) REFERENCES DimCustomer(CustomerId),
  FOREIGN KEY (ProductId) REFERENCES DimProduct(ProductId),
  FOREIGN KEY (SaleDate) REFERENCES DimDate(Date)
);
''')

In [167]:
mycursor.execute('''
INSERT INTO FactSales (OrderRef, CustomerId, ProductId, Quantity, SoldPrice, SalesAmount, SaleDate)
VALUES
  ('ORDER001', 201, 101, 2, 25.99, 51.98, '2023-01-15'),
  ('ORDER002', 202, 102, 3, 15.50, 46.50, '2023-02-20'),
  ('ORDER003', 203, 103, 1, 30.75, 30.75, '2023-03-25');
''')

In [168]:
mycursor.execute('''
SELECT * 
FROM DimDate 
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM DimProduct 
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM DimCustomer 
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

mycursor.execute('''
SELECT * 
FROM FactSales  
''')

result = mycursor.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in mycursor.description])
display(df)

,Date,Day,Month,Quarter,Year
0,2023-01-15,Monday,January,Q1,2023
1,2023-02-20,Wednesday,February,Q1,2023
2,2023-03-25,Saturday,March,Q1,2023


,ProductId,Title,Author,Category
0,101,Book A,Author X,Fiction
1,102,Book B,Author Y,Non-Fiction
2,103,Book C,Author Z,Science


,CustomerId,CustomerName,CustomerType,City
0,201,Customer X,Regular,New York
1,202,Customer Y,VIP,Los Angeles
2,203,Customer Z,Regular,Chicago


,OrderRef,CustomerId,ProductId,Quantity,SoldPrice,SalesAmount,SaleDate
0,ORDER001,201,101,2,25.99,51.98,2023-01-15
1,ORDER002,202,102,3,15.50,46.50,2023-02-20
2,ORDER003,203,103,1,30.75,30.75,2023-03-25


In [169]:
# Close the MySQL connection
mydb.close()